# Classification Process

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


#Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

import time

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br></p>

In [2]:
def CreateDataFrame(types,subtype, level):
    data = pd.read_csv('../Extraction/'+types+"/"+subtype+"/"+level+'.csv')
    data["Class"] = level
    return data

In [3]:
def LoadFullData(types,subtype):
    if(subtype=="All"):
        frames = [CreateDataFrame(types,"Roof","Clean"), 
              CreateDataFrame(types,"Roof","Dirty (0-20)"), 
              CreateDataFrame(types,"Roof","Dirty (20-40)"), 
              CreateDataFrame(types,"Roof","Dirty (40-60)"),
              CreateDataFrame(types,"Roof","Dirty (60-80)"),
              CreateDataFrame(types,"Roof","Dirty (80-100)"),
              CreateDataFrame(types,"Grass","Clean"), 
              CreateDataFrame(types,"Grass","Dirty (0-20)"), 
              CreateDataFrame(types,"Grass","Dirty (20-40)"), 
              CreateDataFrame(types,"Grass","Dirty (40-60)"),
              CreateDataFrame(types,"Grass","Dirty (60-80)"),
              CreateDataFrame(types,"Grass","Dirty (80-100)")]
    else:
        frames = [CreateDataFrame(types,subtype,"Clean"), 
              CreateDataFrame(types,subtype,"Dirty (0-20)"), 
              CreateDataFrame(types,subtype,"Dirty (20-40)"), 
              CreateDataFrame(types,subtype,"Dirty (40-60)"),
              CreateDataFrame(types,subtype,"Dirty (60-80)"),
              CreateDataFrame(types,subtype,"Dirty (80-100)")]
    df = pd.concat(frames)
    #Randomize
    df = shuffle(df)
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    f = df[['FileAddress']].copy()
    del X['FileAddress']
    return X,y,f

In [4]:
def LoadMinimizedData(types,subtype, count):
    df = pd.read_csv('../Selection/Selection-'+types+"-"+subtype+"-"+count+".csv")
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    f = df[['FileAddress']].copy()
    del X['FileAddress']
    return X,y,f

In [5]:
def loader(types,subtype,datasize):
    if(datasize=="Full"):
        X,y,f = LoadFullData(types,subtype)
    else:
        count = datasize.split('-')[-1]
        X,y,f = LoadMinimizedData(types,subtype,count)
    return X,y,f

# Classifications

## Optimize KNN Classfiers

In [6]:
def optimizeKNN(X_train, y_train,cv):
    param_grid = [{'clf__weights': ["distance"], 'clf__n_neighbors': [5]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', KNeighborsClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    end = time.time()
    return grid_search

## Optimize SVM Classfiers

In [7]:
def optimizeSVM(X_train, y_train,cv):
    param_grid = [
            {'clf__kernel': ['poly'],'clf__degree':[1,5],'clf__coef0':[5,10]}
            #,{'clf__kernel': ['rbf'],'clf__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]}
            #,{'clf__kernel': ['sigmoid'],'clf__coef0':[1,5,10,15,20,30]}
            #,{'clf__kernel':['linear'],'clf__C':[1,5,10]}
            ]

    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', SVC())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Optimize Random Forest Classfiers

In [8]:
def optimizeRandomForest(X_train, y_train,cv):
    param_grid = [
            {'clf__n_estimators': [30],'clf__max_features':['auto']},
            ]

    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', RandomForestClassifier())])
        
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Optimize NN Classifiers 

In [9]:
def optimizeNN(X_train, y_train,cv):
    param_grid = [{'clf__hidden_layer_sizes': [(100,50,30,)]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', MLPClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Cycle Work

In [10]:
def cycleWork(X_train, X_test, y_train, y_test,optimize,cv):
    gridsearch = optimize(X_train, y_train,cv)
    traintime = gridsearch.cv_results_['mean_fit_time'].mean()*1000
    print("Mean Fit Time", traintime)
    print("Mean Score Time", gridsearch.cv_results_['mean_score_time'].mean()*1000)

    #print(gridsearch.best_params_)
    estimator = gridsearch.best_estimator_
    params = estimator.get_params()['clf']
    print(params)
    y_train_pred = estimator.predict(X_train)
    trnscore = accuracy_score(y_train, y_train_pred)
    start = time.time()
    y_tst_pred = estimator.predict(X_test)
    end = time.time()
    diff = end-start
    clfperms = X_train.shape[0]/((diff*1000)+0.000000001)
    tstscore = accuracy_score(y_test, y_tst_pred)
    cm = confusion_matrix(y_test, y_tst_pred)
    true_pos = np.diag(cm) 
    precision = np.sum(true_pos / np.sum(cm, axis=0))
    recall = np.sum(true_pos / np.sum(cm, axis=1))
    f1 = 2 * ((precision* recall)/(precision + recall))
    #f1None = f1_score(y_test, y_tst_pred, average=None)
    #f1micro = f1_score(y_test, y_tst_pred, average='micro')
    return [trnscore, tstscore, f1,traintime,clfperms,cm,params,estimator]

In [11]:
def saveresultlist(types,subtype,datasize,clf,iteration,split,classifier,X,y,files):
    data = []
    #print(files.shape[0])
    for i in range(0,files.shape[0]):
        #print(files.iloc[[i]])
        #print(X.iloc[[i]])
        pred = classifier.predict(X.iloc[[i]])
        #print(pred)
        data.append([files.iloc[[i]].values[0][0],y.iloc[[i]].values[0][0],pred[0]])
    numpyarray = np.array(data)
    df = pd.DataFrame(numpyarray, columns = ["File","Y","Pred"])
    fileName = "Real-"+types+'-'+ subtype+"-"+ datasize+"-"+ str(clf)+"-"+ str(iteration)+"-"+ str(split)+'.csv'
    print(type(fileName))
    df.to_csv(fileName,index = False)

In [15]:
def AllWork(types,subtype,sizeDict, cycles = 5, out=False,cross_validation=5):    
    data = []
    for datasize in sizeDict:
        print(datasize)
        classifiersDict = {
                        "KNN":optimizeKNN,
                        "SVM":optimizeSVM,
                        "RF":optimizeRandomForest,
                        "NN":optimizeNN
                          }
        for clf in classifiersDict:
            for i in range(0,cycles):
                X,y,f = loader(types,subtype,datasize)
                #X,y = LoadMinimizedData(types)
                skf = StratifiedKFold(n_splits=cross_validation,shuffle = True)
                j = 0
                for train_index, test_index in skf.split(X, y.values.ravel()):
                    print("Size ",datasize, " Algorithm ",clf, " Trial ",i, " Split ", j," :")
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                    acc = cycleWork(X_train, X_test, y_train, y_test,classifiersDict[clf],cross_validation)
                    #saveresultlist(types,subtype,datasize,clf,i,j,acc[7],X,y,f)
                    print("Train Accuracy ",acc[0],", Test Accuracy",acc[1], "F1 ",acc[2])
                    data.append([clf,datasize.split('-')[-1],i,j,acc[0],acc[1],acc[2],acc[3],acc[4],acc[5],acc[6]])
                    j = j+1
                    #break
                    print("########################################")
    numpyarray = np.array(data)
    df = pd.DataFrame(numpyarray, columns = ["Classifier","Count","Cycle","Split","Train Acc", "Test Acc", "F1 Score","Train Time (ms)","clfs per ms","Confusion Matrix","Params"])
    if(out):
        df.to_csv("All-"+types+'-'+subtype+'.csv',index = False)
    df.head(10)

In [86]:
types = "Cropped"
subtype = "All"
cycles = 1
sizeDict = {
                "Full":None,
                "Min-10":10,
                #"Min-20":20,
                #"Min-30":30,
                #"Min-40":40
                }
AllWork(types,subtype,sizeDict,cycles = cycles, out=True,cross_validation=10)

Full
Size  Full  Algorithm  KNN  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.5s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.8s finished


Mean Fit Time 80.4480791091919
Mean Score Time 96.55022621154785
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
<class 'str'>
Train Accuracy  1.0 , Test Accuracy 0.9568345323741008 F1  5.7575367633416334
Size  Full  Algorithm  SVM  Trial  0  Split  0  :
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   52.1s finished


Mean Fit Time 2530.52419424057
Mean Score Time 147.67592549324033
SVC(C=1.0, cache_size=200, class_weight=None, coef0=10,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
<class 'str'>
Train Accuracy  1.0 , Test Accuracy 0.9892086330935251 F1  5.928915653441464
Size  Full  Algorithm  RF  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Mean Fit Time 15300.212216377258
Mean Score Time 88.40038776397705
SVC(C=1.0, cache_size=200, class_weight=None, coef0=10,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
<class 'str'>
Train Accuracy  0.955667001003009 , Test Accuracy 0.9442446043165468 F1  5.672741211996902
Size  Min-10  Algorithm  RF  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   10.2s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.6s finished


Mean Fit Time 973.8504648208618
Mean Score Time 21.749329566955566
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
<class 'str'>
Train Accuracy  0.9997993981945837 , Test Accuracy 0.9514388489208633 F1  5.7238255891273395
Size  Min-10  Algorithm  NN  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.1min remaining:   29.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished


Mean Fit Time 24642.14835166931
Mean Score Time 9.051084518432617
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
<class 'str'>
Train Accuracy  0.9199598796389168 , Test Accuracy 0.9136690647482014 F1  5.522158985336111


In [87]:
types = "Solar Data"
cycles = 1
subtype = "All"
sizeDict = {
                "Full":None,
                "Min-10":10,
                #"Min-20":20,
                #"Min-30":30,
                #"Min-40":40
                }
AllWork(types,subtype,sizeDict,cycles = cycles, out=True,cross_validation=10)
#subtype = "Grass"
#AllWork(types,subtype,sizeDict,cycles = cycles, out=True,cross_validation=10)
#subtype = "Roof"
#AllWork(types,subtype,sizeDict,cycles = cycles, out=True,cross_validation=10)

Full
Size  Full  Algorithm  KNN  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.9s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.6s finished


Mean Fit Time 83.29906463623047
Mean Score Time 74.79853630065918
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
<class 'str'>
Train Accuracy  1.0 , Test Accuracy 0.9910233393177738 F1  5.9460691387263855
Size  Full  Algorithm  SVM  Trial  0  Split  0  :
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   49.0s finished


Mean Fit Time 2006.5865218639374
Mean Score Time 148.72605204582214
SVC(C=1.0, cache_size=200, class_weight=None, coef0=5,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
<class 'str'>
Train Accuracy  1.0 , Test Accuracy 0.9946140035906643 F1  5.966384683251594
Size  Full  Algorithm  RF  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Mean Fit Time 18472.924798727036
Mean Score Time 95.44968008995058
SVC(C=1.0, cache_size=200, class_weight=None, coef0=10,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
<class 'str'>
Train Accuracy  0.9837642814191221 , Test Accuracy 0.9784560143626571 F1  5.878206993532988
Size  Min-10  Algorithm  RF  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   11.6s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.6s finished


Mean Fit Time 829.1986465454102
Mean Score Time 15.499520301818848
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
<class 'str'>
Train Accuracy  0.9995991180597315 , Test Accuracy 0.9533213644524237 F1  5.751178654774275
Size  Min-10  Algorithm  NN  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   57.0s remaining:   24.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.1min finished


Mean Fit Time 21413.498854637146
Mean Score Time 9.101104736328125
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
<class 'str'>
Train Accuracy  0.9428743235117258 , Test Accuracy 0.9317773788150808 F1  5.608811393439282


In [16]:
types = "NoBG"
cycles = 5
subtype = "All"
sizeDict = {
                "Full":None,
                "Min-10":10,
                "Min-20":20,
                "Min-30":30,
                "Min-40":40
                }
AllWork(types,subtype,sizeDict,cycles = cycles, out=True,cross_validation=10)

Full
Size  Full  Algorithm  KNN  Trial  0  Split  0  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.0s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.6s finished


Mean Fit Time 52.55095958709717
Mean Score Time 60.15036106109619
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
Train Accuracy  1.0 , Test Accuracy 0.9712230215827338 F1  5.839652429476197
########################################
Size  Full  Algorithm  KNN  Trial  0  Split  1  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.4s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.0s finished


Mean Fit Time 53.84988784790039
Mean Score Time 58.10110569000244
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
Train Accuracy  1.0 , Test Accuracy 0.9766187050359713 F1  5.86211458000402
########################################
Size  Full  Algorithm  KNN  Trial  0  Split  2  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Mean Fit Time 6838.601636886597
Mean Score Time 6.048822402954102
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Train Accuracy  0.9905773857257418 , Test Accuracy 0.9819168173598554 F1  5.892615257897031
########################################
Size  Min-40  Algorithm  NN  Trial  4  Split  8  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   22.3s remaining:    9.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   28.1s finished


Mean Fit Time 7511.150360107422
Mean Score Time 5.900359153747559
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Train Accuracy  0.9957907396271798 , Test Accuracy 0.9855072463768116 F1  5.923897671034738
########################################
Size  Min-40  Algorithm  NN  Trial  4  Split  9  :
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   23.2s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.1s finished


Mean Fit Time 7384.149718284607
Mean Score Time 6.6009521484375
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Train Accuracy  0.98937875751503 , Test Accuracy 0.9782214156079855 F1  5.865768939789033
########################################
